# Tutorial 1: Data Preprocessing

---

### Introduction

Welcome! This tutorial will show you how to visualise and preprocess astronomical data using python. From this tutorial, you will learn the following:

1. How to download astronomical data
2. How to read and visualise data
3. How to preprocess imagery data

The data [GalaxyMNIST](https://github.com/mwalmsley/galaxy_mnist) contains 10,000 images of galaxies (either 3x64x64 or 3x224x224), labelled by Galaxy Zoo volunteers as belonging to one of four morphology classes, where the classes are:

0. smooth and round
1. smooth and cigar-shaped
2. edge-on-disk
3. unbarred spiral

The galaxies are selected from `Galaxy Zoo DECaLS Campaign` A (GZD-A). The images are shown to volunteers on Galaxy Zoo for them to classify. For each image, at least 17 people were asked the necessary questions, and at least half of them must have answered with the given class. The class labels are, therefore, much more confident than, for example, simply labelling with the most common answer to some question. For more info, visit this [link](https://github.com/mwalmsley/galaxy_mnist)


Labelling data is a big topic in machine learning. Labelled data allow us to use supervised machine learning tools to train on labelled data and classify unlabeled ones. However, labelling data can be costly. Scientists have been trying to figure out a way to label data, but the easiest is that we can depend on human knowledge to label data like the `Galaxy Zoo DECaLS Campaign`.  However, unlabeled data can also be of some benefit if analyzed with the right tools, like unsupervised machine learning. Those tools allow us to explore (find patterns) and cluster the data, leading to discoveries in the data.


---

### Downloading data

First, let us call the python script that will let us download the data.

In [ ]:
# https://github.com/mwalmsley/galaxy_mnist
from galaxy_mnist import GalaxyMNISTHighrez

'GalaxyMNISTHighrez' class has the following attributes: 
 - `root`: Specifing the target directory to download the data
 - `download:` A boolean value, `True` to download the images
 - `train:` A boolean value, `True` to download just the training data, `False` will download just the testing data.
   - `Notice:` The data has a fixed 80/20 train/test division.

In [ ]:
# for the training data
dataset_train = GalaxyMNISTHighrez(
    root='data_import/data',
    download=True,
    train=True  # by default True, or False for canonical test set
)
# for the testing data
dataset_test = GalaxyMNISTHighrez(
    root='data_import/data',
    download=True,
    train=False  # by default True, or False for canonical test set
)

---

### Reading data

First, let's read the inputs `images` and the outputs `labels` into two seperate variables.

In [ ]:
images_train = dataset_train.data
labels_train = dataset_train.targets

Now check the shapes and data types to see they are consistent.

In [ ]:
print("Shape of the input (training): "+str(images_train.shape)+ ", type: "+ str(images_train.dtype))
print("Shape of the output (training): "+str(labels_train.shape)+ ", type: "+ str(labels_train.dtype))



We notice the following:
 1. The input data has 8000 samples with 3 channels, which stand for `rgb` colors, where each sample has a size of 224 x 224.
 2. The output is a one-dimensional array which contains the labels of the samples. Each label is 0,1,2,3 according to the categories listed in the first cell above. 

**Exercise 1:**
Store the testing set inputs and outputs into two different variables, and display their shapes and data types.

In [ ]:
### -- Code here --



Now, let's check the frequencies of the labels/output. First we display the output.

In [ ]:
GalaxyMNISTHighrez.classes

   Where (as indicated in the first cell):
   - 0: `smooth_round`,
   - 1: `smooth_cigar`,
   - 2: `edge_on_disk`,
   - 3: `unbarred_spiral`

Next use numpy functions to summarize the number in each class

In [ ]:
import numpy as np # For handling N-DIMENSIONAL ARRAYS

In [ ]:
print(labels_train)
labels_trainNP = np.array(labels_train)
unique_labels, count_labels = np.unique(labels_trainNP, return_counts=True)
print("Label, count:", unique_labels, count_labels )

### Data visualisation

We can also try to visualise label frequencies in a bar graph. To do this, we need the `pyplot' graphics package.

In [ ]:
import matplotlib.pyplot as plt #This displays graphs once they have been created

The code for plotting is below. To make a clear display, we replace class labels with the actual class names.

In [ ]:
plt.bar(range(len(unique_labels)), count_labels, align='center')
plt.xticks(range(len(unique_labels)), GalaxyMNISTHighrez.classes)
plt.show()

**Exercise 2:**
Count the class frequencies for the testing data, and visualise using a histogram.

In [ ]:
### -- Code here --


Now let us take a look at the images of the different galaxy shapes.

In [ ]:
from PIL import Image #  PIL is the Python Imaging Library

In [ ]:
# Display nImage images for each class
nImage = 5
# Loop through classes
for j in range(len(GalaxyMNISTHighrez.classes)):
    fig = plt.figure(figsize=(8, 8))# Figure is 8 inches by 8 inches
    for i in range (nImage):    # Create images in each column
        train_image = images_train[(labels_train == j)][i]
        fig.add_subplot(1, nImage, i+1)
        plt.imshow(np.array(train_image).transpose(1, 2, 0))
    print("label: "+str(GalaxyMNISTHighrez.classes[j]))
    plt.tight_layout()
    plt.show() 

The resolution between classes seems unclear. For instance, `smooth cigar` and `edge_on_disk` look like each other. this will decrease accuracy when performing classification or clustering.

---

### Data preprocessing

Classical machine learning often subjects the data to extensive preprocessing before classification, especially when the datasets involved are relatively small. Preprocessing functions include feature extraction, optimization, normalization, augmentation, cropping, resizing, denoising, and so on. For each of these functions, there are many alternative algorithms. 

On the other hand, deep learning models with lots of data are often referred to as End-to-End learning models ([E2E](https://towardsdatascience.com/e2e-the-every-purpose-ml-method-5d4f20dafee4)), since they can extract information from raw data.

In the following tutorial, we will use an [_autoencoder_](https://en.wikipedia.org/wiki/Autoencoder), which uses deep learning to extract features. In theory, we can feed the raw data into the autoencoder without any preprocessing. However, in practice this greatly increases the computation time required. In this tutorial series we need to train the models in a reasonable time so that we can bring the idea across. The issue is especially important because optimal use of raw data requires hyperparameter tuning, which is even more time consuming.

However, preprocessing has a caveat. We need to make sure that the preprocessing does not lose any useful information. How can we verify this? In our case, since the data is imagery data we can examine the preprocessed images visually. If we can still easily classify them, then we can conclude that the preprocessing is effective.

Using an autoencoder is just one possibility. Participants are encouraged to explore alternatives.

In this tutorial, we will try to reduce the complexity of the data by doing the following:
   - Convert images to grayscale (reduces data size by a factor  of 3)
   - Reduce the size of the image by a factor of 4.

#### A. Grayscaling
We begin with conversion to grayscale, following this [tutorial](https://www.tutorialspoint.com/pytorch-how-to-convert-an-image-to-grayscale) from [tutorialspoint.com](https://www.tutorialspoint.com).

In [ ]:
import torchvision.transforms as transforms #Transforms are for common image transformations

In [ ]:
# Display the shape of image data
images_train.shape

We can see that the original image size has three channels which stand for the RGB colours. Now let us perfom the grayscalling.

In [ ]:
transformGrey = transforms.Grayscale()
images_trainGrey = transformGrey(images_train)

Now let us check the size of the resulting tensor

In [ ]:
images_trainGrey.shape

we can see the number of channels went down from 3 to 1

**Exercise 3:**
Display a set grayscale images for each classes (like the display of images above)

In [ ]:
### -- Code here__


We can notice that visually that nothing changed much from the original data. However, we reduced the complexity of the data by a favour of 3, which is enormous!

#### B. Image resizing  
(see  [this reference](https://www.tutorialspoint.com/pytorch-how-to-resize-an-image-to-a-given-size))

Reducing the size of the image can further reduce the complexity of the image data. However, we must be careful not to reduce so much as to removed distinctions between classes.

In [ ]:
# Define the transform. The number in parenthesis gives number of pixels per side.
transformResize = transforms.Resize(56)

Now let's reduce the grayscale images:

In [ ]:
images_trainResized = transformResize(images_trainGrey)

Check the size again:

In [ ]:
images_trainResized.shape

**Exercise 4:**
Visualise the resized images for all classes

In [ ]:
#___ Code here___





##### **_End of the pre-processing pipeline_**

Now the above pre-processing is not the only way where we can reduce the features, the following can also be done:
1. Cropping images away from the centre
2. Image segmentation, maybe to remove background noise (Deep learning can be used for this)
3. Explore ...

### Normalization

The pixel levels run from 0 to 255. If we divide by 255, we will put the levels within a standard scale of 0 to 1. Then they are suitable to use with other features that are similarly normalised. ([source](https://developers.google.com/machine-learning/data-prep/transform/normalization)).

In [ ]:
images_trainPre = images_trainResized/255

Now let us check the values for one image before and after the nomalisation.

In [ ]:
print("Pixel levels before normalization:")
print(images_trainResized[0][0][0])
print("Pixel levels after  normalization:")
print(images_trainPre[0][0][0])

**Exercise 5**: Perform the preprocessing pipeline but on the testing set

In [ ]:
#___ Code here___


---

### **_End of Tutorial 1_**